# Python Workshop Day 2 
Welcome to the Chemistry Department Programming Workshop Day 2!  
   
These tutorials will focus on more advanced data analysis and visualization tools, and give some time for integration with your own data.
  
Day 2 consists of 4 sub-modules:  
* [`2.0 Data Processing and Analysis`](./2.0_day2_data_analysis_and_stats.ipynb) – statistical analysis, regression and smoothing  
* [`2.1 Complex Formatting and Data Visualization`](./2.1_day2_complex_formatting.md) – using class strutures to access complex formatting 
* [`2.2 Prompt Engineering for Generative AI`](./2.2_day2_prompt_engineering_for_generative_ai.md) – how to write effective prompts for GPT and interpret the code it writes
* [`2.3 Applications to Your Data`](./2.3_day2_applications_to_your_data.md) – try to apply workshop tools to you own datasets!

## 2.0 Data Processing and Analysis
---

**Contents**  
  
In this module, you will complete:
 * [2.0.1 Simple Linear Regression](#211-interpreting-class-structures)
 * [2.0.2 Add Error Bars to Matplotlib Plots](#212-writing-a-custom-plotting-class)
 * [2.0.3 Polynomial Curve Fitting](#212-writing-a-custom-plotting-class)
 * [2.0.4 Gaussian Process Regression](#212-writing-a-custom-plotting-class)




## 2.0.1 Simple Linear Regression
  
In this section, we will use the popular data science/machine learning package [**scikit-learn**](https://scikit-learn.org/stable/). If you do not already have it added to your python environment add it now.  
 
**scikit-learn** has many nice built-in functions for regression, dimensionality reduction (see [PCA](https://www.nature.com/articles/s43586-022-00184-w#:~:text=Principal%20component%20analysis%20is%20a,variance%20of%20all%20the%20variables.)), clustering, and image processing, among other methods. Note also that you can perform the same actions using [scipy's implementation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.linregress.html), or code the actual matrix operations yourself using numpy.   
  
Let's first create some sample data:
  


In [ ]:
from sklearn.linear_model import LinearRegression
import numpy as np
import matplotlib.pyplot as plt

x_data = np.linspace(start=-20, stop=25, num=100)
y_data = x_data**3 + 2000*np.sin(x_data)
plt.plot(x_data,y_data)

And now we will fit the data using scikit-learn linear regressor. The `.predict` method works for all scikit learn `Regressor()` type objects, and is one way they remain internally consistent.

In [ ]:
my_linreg = LinearRegression()

my_linreg.fit(x_data.reshape(-1,1), 
              y_data.reshape(-1,1))

y_estim = my_linreg.predict(x_data.reshape(-1,1))

fig, ax = plt.subplots()
ax.plot(x_data, y_estim,'r-', label="Linear Estimate")
ax.plot(x_data, y_data, 'teal', label="Original Function")

ax.legend()

We can also get the Pearson Correlation coefficient (R squared) using scikit-learn. Note that there are other types of correlation coefficients that may be more appropriate for your data, and beware [*Anscombe's Quartet*](https://en.wikipedia.org/wiki/Anscombe%27s_quartet).

In [ ]:
from sklearn.metrics import r2_score

r2 = r2_score(y_data, y_estim)

# here we use an 'f-string' to print the data
# f-strings allow us to specify formatting
# here we use ":.3" to specify a float with 3 decimal places
print(f"The Pearson Correlation Coefficient is: \t {r2:.3}")

Let's add this information to the plot as legend text:

In [ ]:
handles, labels = ax.get_legend_handles_labels()
labels[0] = f"Linear Estimate $R^2$ = {r2:.3}"
ax.legend(labels = labels, handles = handles)
fig

## 2.0.2 Add Error Bars to Matplotlib Plots
  
In the next example, we will use matplotlib's built in tools to add errorbars to each of our sample data points. We will later shade error regions when we do Gaussian Process Regression (2.0.4). We can also revisit this example and fit a line to our noisy data after section 2.0.3.
  
First we define a function for our example data:
  

In [ ]:
# Define a function to fit that is the "True" value we don't know. 
def my_func(x, a, b):
    """
    Always make a description for your function. 
    
    args: 
        a: a constant multiplier
        b: The y intercept
        
    returns: 
        The function evaluated at x locations. 
    """
    return a * x**3 + b

Then add some noise to our random data:

In [ ]:
# Generate some random data
np.random.seed(0)
x = np.linspace(start=0, 
                stop=10, 
                num=30)

y = 3 * x**3 + 2 # Invent random values for a and b for the "True" fxn

noise_amt = .2
added_noise = abs(np.random.normal(scale=y*noise_amt, size=x.size))

And finally plot the results:

In [ ]:
# Plot the data with error bars

plt.errorbar(x, 
             y+added_noise, 
             yerr=added_noise, 
             fmt='o', 
             label='Data with error bars', 
             ecolor='r', 
             capsize=5)

# Add labels and legend
plt.xlabel('X')
plt.ylabel('Y')
plt.legend()
plt.title('Random Data with Error Bars')

plt.text(x=0.5, 
         y=2000, 
         s="Annotation for when you want it.")

# Show the plot
plt.show()

## 2.0.3 Polynomial Curve Fitting
  
Here we use scipy instead of scikit-learn to fit a polynomial to noisy sine data. If you want to instead fit a custom function (like Michelis-Menten Kinetics or a Langmuir Isotherm for example), you can tweak your guess function. Alternative curve fitting algorithms such as [universal functions originators](https://en.wikipedia.org/wiki/Symbolic_regression) and [genetic algorithms](https://gplearn.readthedocs.io/en/stable/) are also fun to play around with.
  
First we define the functions for our example data:

In [ ]:
import numpy as np
from scipy.optimize import curve_fit
import pandas as pd

def Some_Unknown_Function1(x, a, b=5):
    """
    Modified sine function. 
    
    args: 
        x (float); Independent Variable Vals
        a (float); Controls amplitude of sine()
        b (float); Controls y-offset of function. 
    """
    func_value = b + a*np.sin(x)
    return func_value

In [ ]:
def Some_Unknown_Function2(x, noiseval=0):
    """
    X squared function with added noise. 
    
    args: 
        x (float); Independent Variable Vals
        noiseval (zero or positive float); determines stdev of Gaussian noise. 
    """
    y = x**2
    noise = np.random.normal(loc=0, 
                             scale=noiseval, 
                             size=len(x))
    return y + noise

And create the y values...

In [ ]:
x_vals = np.linspace(start=0, 
                     stop=10, 
                     num=55)

y_vals = Some_Unknown_Function1(x_vals, 
                                a=3, 
                                b=15)

y_vals2 = Some_Unknown_Function2(x_vals, 
                                 noiseval=15)

In [ ]:
plt.plot(x_vals, 
         y_vals2, 'b.', 
         label="Noisy Data")

plt.plot(x_vals, 
         Some_Unknown_Function2(x_vals, 
                                noiseval=0), 
         'k-', 
         label="No Noise Added")

plt.legend()

The `curve_fit()` method from scipy is used here to get teh function parameters. Note that on real data, you will already have your x and y values. You will then need to identify a functional form written as a python function that can be passed to the `curve_fit()` method. Note also that there may not be a unique solution, and different optimization algorithms may yield different parameters. Further, the same optimization algorithm may yield different results when used on the same data following a log transformation. Be careful when fitting curves, and read the [documentation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html) for more information.

In [ ]:
y_estim = curve_fit(Some_Unknown_Function2, 
                    xdata=x_vals, 
                    ydata=y_vals2)
# Returns function parameters. Saved as a tuple of popt and pcov - see the documentation to interpret these values
y_estim

Now that we have our parameters, we need to check that our fit is reasonable:

In [ ]:
plt.plot(x_vals, 
         x_vals**2, 
         'b.', 
         label="True Function")

plt.plot(x_vals, 
         y_vals2, 
         'g*',
         label="Noisy Measurements")

plt.plot(x_vals, # x points
         Some_Unknown_Function2(x_vals, *y_estim[1]), 
         'r-', # Color and line settings
         label="Reconstructed Function"
        )

plt.text(x=5, # X location of text
         y=-10, # Y location of text
         s=f"y popt estimate: {np.round(y_estim[0],4)}\ncov(popt) estimate: {np.round(y_estim[1],4)}"
        )
# popt; i.e.: how many optimal parameters being searched. 
# cov; i.e.: the covariance of popt. See documentation. 
plt.legend()

On your own, see if you can fit a curve to the noisy data from section 2.0.2 above.

## 2.0.4 Gaussian Process Regression
  
Gaussian Process Regression is another technique applied to curve-fitting. This method is on the 'machine learning' end of the spectrum between datascience/statistics and AI. It can be useful for low dimensional datasets, and also neatly demonstrates error region shading. For more information in Gaussian Processes and the important assumptions and use cases, see this [link](https://scikit-learn.org/stable/modules/gaussian_process.html).  
   
For now, we'll skip some of the background and get straight to the demonstration:

In [ ]:
from sklearn.datasets import make_friedman2
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel

X = np.linspace(start=0, stop=10, num=1000).reshape(-1, 1)
y = np.squeeze(X * np.sin(X))

plt.plot(X, y, label=r"$f(x) = x \sin(x)$", linestyle="dotted")
plt.legend()
plt.xlabel("$x$")
plt.ylabel("$f(x)$")
_ = plt.title("True process data", fontsize=16)

With our sample data generated, we will proceed to making a Gaussian Process `Regressor()` object:

In [ ]:
rng = np.random.RandomState(1)
training_indices = rng.choice(np.arange(y.size), size=6, replace=False)
X_train, y_train = X[training_indices], y[training_indices]

kernel = 1 * RBF(length_scale=1.0, 
                 length_scale_bounds=(1e-2, 1e2))

gaussian_process = GaussianProcessRegressor(kernel=kernel, 
                                            n_restarts_optimizer=9)

gaussian_process.fit(X_train, 
                     y_train)

gaussian_process.kernel_

Similar to the simple linear model, we will use the `predict()` method:

In [ ]:
mean_prediction, std_prediction = gaussian_process.predict(X, return_std=True)

And now we will display the data using shaded error regions:

In [ ]:
fig, ax = plt.subplots(nrows=1, 
                       ncols=1, 
                       figsize=(10,10))

# Plot original data. 
ax.plot(X, 
        y, 
        label=r"$f(x) = x \sin(x)$", 
        linestyle="dotted")

# Plot noisy data that was subsampled for training set (if using machine learning)
ax.scatter(X_train, 
           y_train, 
           label="Observations")

# Plot mean line from model or curvefit or whatnot. 
ax.plot(X, 
        mean_prediction, 
        label="Mean prediction")

# MAKE SOLID ERROR REGION
ax.fill_between(
    X.ravel(), # Flatten array so it plots. 
    mean_prediction - 1.96 * std_prediction, # Define lower 95% CI limit
    mean_prediction + 1.96 * std_prediction, # Define upper 95% CI limit
    alpha=0.3, # how opaque do you want the error bars (%). 
    color="gray",
    label=r"95% confidence interval", # Label the shaded region for inclusion in legend
)

# Plot decorations (i.e.: legend, labels, title, etc.)
ax.legend()
ax.set_xlabel("$x$")
ax.set_ylabel("$f(x)$")
_ = ax.set_title("Gaussian process regression on noise-free dataset", fontsize=20)

Note again!  
There is a difference between the confidence interval and the prediction interval. Be sure to [read up on this](https://towardsdatascience.com/how-confidence-and-prediction-intervals-work-4592019576d8) before using either one. When calculating either interval, you make fundamental assumptions about the distribution of your data- ensure that these assumptions are appropriate!


---
### Congratulations!  
You have now completed [**2.0 Data Processing And Analysis**]().  
Please proceed to [**2.1 Complex Formatting and Data Visualization**](./2.1_day2_complex_formatting.md).